In [1]:
import psycopg2
import pandas as pd
import os
import psycopg2
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel

df = pd.read_csv('GamingToxicPhrases.csv')
df = df.replace('Toxic', True)
df = df.replace('Not Toxic', False)
print(df.info())
df.rename(columns={'text': 'sentence', 'is_toxic': 'isToxic'}, inplace=True)
print(df.head())

c:\Users\patri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      250 non-null    object
 1   is_toxic  250 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 2.3+ KB
None
                sentence  isToxic
0  Hang from the ceiling     True
1          Kill yourself     True
2          Go buy a rope     True
3             Get cancer     True
4   I hope your mom dies     True


C:\Users\patri\AppData\Local\Temp\ipykernel_45104\1642148272.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('Not Toxic', False)


In [2]:
csv = pd.read_csv('toxicity_en.csv')
#csv.head()
csv = csv.replace('Toxic', True)
csv = csv.replace('Not Toxic', False)
#print(csv.head())
print(csv.info())
csv.rename(columns={'text': 'sentence', 'is_toxic': 'isToxic'}, inplace=True)
print(csv.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1000 non-null   object
 1   is_toxic  1000 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 8.9+ KB
None
                                            sentence  isToxic
0  Elon Musk is a piece of shit, greedy capitalis...     True
1  The senile credit card shrill from Delaware ne...     True
2  He does that a lot -- makes everyone look good...     True
3                                         F*ck Lizzo     True
4  Epstein and trump were best buds!!! Pedophiles...     True


C:\Users\patri\AppData\Local\Temp\ipykernel_45104\1122102729.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  csv = csv.replace('Not Toxic', False)


In [3]:
#ignore this file. Just run the files that dont have this messagae
from sklearn.model_selection import train_test_split

# Check the data
print(df.info())
print(df.head())

dataset = pd.concat([df, csv])
print(dataset.info())
print(dataset.head())

# Split the data into train and validation sets
train_df, valid_df = train_test_split(dataset, test_size=0.2, random_state=42)

# Ensure labels are integers (convert `isToxic` from bool to int)
train_df['isToxic'] = train_df['isToxic'].astype(int)
valid_df['isToxic'] = valid_df['isToxic'].astype(int)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  250 non-null    object
 1   isToxic   250 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 2.3+ KB
None
                sentence  isToxic
0  Hang from the ceiling     True
1          Kill yourself     True
2          Go buy a rope     True
3             Get cancer     True
4   I hope your mom dies     True
<class 'pandas.core.frame.DataFrame'>
Index: 1250 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1250 non-null   object
 1   isToxic   1250 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 20.8+ KB
None
                sentence  isToxic
0  Hang from the ceiling     True
1          Kill yourself     True
2          Go buy a rope     True
3             Get cancer     True
4   I hop

In [4]:
#ignore this file. Just run the files that dont have this messagae
from transformers import BertTokenizer
import torch

class BERTDataset:
    def __init__(self, texts, labels, max_len=128):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        tokenized_text = self.tokenizer(
            text,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True,
        )
        ids = tokenized_text['input_ids']
        mask = tokenized_text['attention_mask']
        token_type_ids = tokenized_text['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(label, dtype=torch.long)
        }


In [5]:
#ignore this file. Just run the files that dont have this messagae
from torch.utils.data import DataLoader

# Create datasets
train_dataset = BERTDataset(
    texts=train_df['sentence'].values,
    labels=train_df['isToxic'].values
)

valid_dataset = BERTDataset(
    texts=valid_df['sentence'].values,
    labels=valid_df['isToxic'].values
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


c:\Users\patri\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\patri\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
#ignore this file. Just run the files that dont have this messagae
import torch.nn as nn
class ToxicModel(nn.Module):
    def __init__(self):
        super(ToxicModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased", return_dict=False)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, token_type_ids, attention_mask, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits



In [7]:
#ignore this file. Just run the files that dont have this messagae
def train_model(self, train_dataset, valid_dataset, epochs=1):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)

        param_optimizer = list(self.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_paramaters = [
            {
                'params' : [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay': 0.01,
            },
            {
                'params' : [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
            },
        ]

        optimizer = torch.optim.Adam(optimizer_grouped_paramaters, lr=5e-5)
        criterion = nn.CrossEntropyLoss()

        for i in range(epochs):
            self.train()
            train_loss = 0
            train_correct = 0
            train_total = 0
            for batch in train_dataset:
                optimizer.zero_grad()
                ids = batch['ids'].to(device)
                mask = batch['mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                target = batch['target'].to(device)

                logits = self(ids, token_type_ids, mask)
                loss = criterion(logits, target)
                loss.backward()
                optimizer.step()

                # Calculate accuracy for training
                _, predicted = torch.max(logits, dim=1)
                train_correct += (predicted == target).sum().item()
                train_total += target.size(0)

                train_loss += loss.item() * batch['ids'].size(0)

            train_loss = train_loss / len(train_dataset)
            train_accuracy = 100 * train_correct / train_total  # Calculate training accuracy

            self.eval()
            valid_loss = 0
            valid_correct = 0
            valid_total = 0
            for batch in valid_dataset:
                ids = batch['ids'].to(device)
                mask = batch['mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                target = batch['target'].to(device)

                logits = self(ids, token_type_ids, mask)
                loss = criterion(logits, target)

                # Calculate accuracy for validation
                _, predicted = torch.max(logits, dim=1)
                valid_correct += (predicted == target).sum().item()
                valid_total += target.size(0)

                valid_loss += loss.item() * batch['ids'].size(0)

            valid_loss = valid_loss / len(valid_dataset)
            valid_accuracy = 100 * valid_correct / valid_total  # Calculate validation accuracy

            print(f"Epoch {i+1}/{epochs}.. Train loss: {train_loss:.3f}.. Train accuracy: {train_accuracy:.2f}%.. Validation loss: {valid_loss:.3f}.. Validation accuracy: {valid_accuracy:.2f}%")


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Rest of your code remains unchanged...

# Split dataset into training and validation sets
train_df, valid_df = train_test_split(csv, test_size=0.2, random_state=42)

# Ensure labels are integers
train_df['isToxic'] = train_df['isToxic'].astype(int)
valid_df['isToxic'] = valid_df['isToxic'].astype(int)

# Create BERT Dataset class
class BERTDataset:
    def __init__(self, texts, labels, max_len=128):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        tokenized_text = self.tokenizer(
            text,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True,
        )
        ids = tokenized_text['input_ids']
        mask = tokenized_text['attention_mask']
        token_type_ids = tokenized_text['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(label, dtype=torch.long)
        }

# Create DataLoaders
train_dataset = BERTDataset(
    texts=train_df['sentence'].values,
    labels=train_df['isToxic'].values
)

valid_dataset = BERTDataset(
    texts=valid_df['sentence'].values,
    labels=valid_df['isToxic'].values
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Define the ToxicModel class
class ToxicModel(nn.Module):
    def __init__(self):
        super(ToxicModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased", return_dict=False)
        self.dropout = nn.Dropout(0.2)  # Increased dropout to prevent overfitting
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, token_type_ids, attention_mask, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Define the training loop with learning rate tuning and class weighting
def train_model(model, train_loader, valid_loader, epochs=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Class weights (if needed)
    # Class weights (increase weight for the minority class)
    class_weights = torch.tensor([0.7, 1.3]).to(device)  # Change these weights based on class distribution
    criterion = nn.CrossEntropyLoss(weight=class_weights)


    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # Lower learning rate for better stability

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        for batch in train_loader:
            optimizer.zero_grad()
            ids = batch['ids'].to(device)
            mask = batch['mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            target = batch['target'].to(device)

            logits = model(ids, token_type_ids, mask)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()

            # Calculate accuracy for training
            _, predicted = torch.max(logits, dim=1)
            train_correct += (predicted == target).sum().item()
            train_total += target.size(0)
            train_loss += loss.item() * batch['ids'].size(0)

        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        valid_loss = 0 
        valid_correct = 0
        valid_total = 0
        valid_pred = []
        valid_true = []

        with torch.no_grad():
            for batch in valid_loader:
                ids = batch['ids'].to(device)
                mask = batch['mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                target = batch['target'].to(device)

                logits = model(ids, token_type_ids, mask)
                loss = criterion(logits, target)

                # Calculate accuracy for validation
                _, predicted = torch.max(logits, dim=1)
                valid_correct += (predicted == target).sum().item()
                valid_total += target.size(0)

                valid_loss += loss.item() * batch['ids'].size(0)
                valid_pred.extend(predicted.cpu().numpy())
                valid_true.extend(target.cpu().numpy())

        valid_loss = valid_loss / len(valid_loader.dataset)
        valid_accuracy = 100 * valid_correct / valid_total

        # Calculate additional metrics
        valid_precision = precision_score(valid_true, valid_pred)
        valid_recall = recall_score(valid_true, valid_pred)
        valid_f1 = f1_score(valid_true, valid_pred)

        print(f"Epoch {epoch+1}/{epochs}.. Train loss: {train_loss:.3f}.. Train accuracy: {train_accuracy:.2f}%.. "
              f"Validation loss: {valid_loss:.3f}.. Validation accuracy: {valid_accuracy:.2f}%.. "
              f"Precision: {valid_precision:.2f}%.. Recall: {valid_recall:.2f}%.. F1: {valid_f1:.2f}%")

# Initialize and train the model
model = ToxicModel()
train_model(model, train_loader, valid_loader, epochs=25)

Epoch 1/25.. Train loss: 0.656.. Train accuracy: 50.00%.. Validation loss: 0.660.. Validation accuracy: 48.00%.. Precision: 0.48%.. Recall: 1.00%.. F1: 0.65%
Epoch 2/25.. Train loss: 0.643.. Train accuracy: 50.50%.. Validation loss: 0.658.. Validation accuracy: 48.00%.. Precision: 0.48%.. Recall: 1.00%.. F1: 0.65%
Epoch 3/25.. Train loss: 0.641.. Train accuracy: 52.38%.. Validation loss: 0.648.. Validation accuracy: 48.00%.. Precision: 0.48%.. Recall: 1.00%.. F1: 0.65%
Epoch 4/25.. Train loss: 0.628.. Train accuracy: 52.38%.. Validation loss: 0.696.. Validation accuracy: 53.00%.. Precision: 0.51%.. Recall: 0.43%.. F1: 0.47%
Epoch 5/25.. Train loss: 0.626.. Train accuracy: 52.12%.. Validation loss: 0.650.. Validation accuracy: 59.00%.. Precision: 0.56%.. Recall: 0.65%.. F1: 0.60%
Epoch 6/25.. Train loss: 0.609.. Train accuracy: 55.00%.. Validation loss: 0.601.. Validation accuracy: 52.50%.. Precision: 0.50%.. Recall: 1.00%.. F1: 0.67%
Epoch 7/25.. Train loss: 0.582.. Train accuracy: 58.

In [11]:
# Assuming the model has already been trained
torch.save(model.state_dict(), 'toxic_model.pth')
print("Model saved to toxic_model.pth")


Model saved to toxic_model.pth
